In [1]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from scipy import spatial
import itertools

Using TensorFlow backend.


In [2]:

padded_embedding = pickle.load( open( "padded_embedding.p", "rb" ) )
#print(padded_embedding[1])


In [3]:
## randomize the list
random.shuffle(padded_embedding)

training_dataset,test_dataset = train_test_split(padded_embedding, train_size=.80, test_size=.20)



In [5]:
## dumping the train and test set


pickle.dump( training_dataset, open( "training_data.p", "wb" ) )

pickle.dump( test_dataset, open( "testing_data.p", "wb" ) )

In [12]:
## In this cell we segrigate inputs and labels for training the model
x, y = list(), list()
nonecount = list()
#print('First padding',a[0])
for k,i in enumerate(training_dataset):
    if isinstance(i,list):
        
        seq_x, seq_y = i[:len(i) - 1], i[-1]
        x.append(seq_x)
        y.append(seq_y)
    else:
        nonecount.append(k)
print(len(nonecount))

# convert the list to numpy array
num_x_train = np.array(x)
num_y_train = np.array(y)

0


In [16]:
## In this cell we segrigate inputs and labels for testing the model
x_test, y_test = list(), list()
nonecount1 = list()
#print('First padding',a[0])
for k,i in enumerate(test_dataset):
    if isinstance(i,list):
        
        seq_x_test, seq_y_test = i[:len(i) - 1], i[-1]
        x_test.append(seq_x_test)
        y_test.append(seq_y_test)
    else:
        nonecount.append(k)
print(len(nonecount1))

# convert the list to numpy array
num_x_test = np.array(x_test)
num_y_test = np.array(y_test)

0


In [21]:
# univariate lstm example
## with sgd optimizer
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers

output_embeddings = []

# choose a number of time steps
n_steps = 9

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 50
x = num_x_train.reshape((num_x_train.shape[0], num_x_train.shape[1], n_features))
y = num_y_train
#print(x)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(50))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mse')
# fit model
model.fit(x, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = num_x_test
x_input = x_input.reshape((num_x_test.shape[0], n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
#print(yhat)
output_embeddings.append(yhat)

In [22]:
## dumping the sample outputs
pickle.dump( output_embeddings, open( "output_embeddings.p", "wb" ) )

In [23]:
## dumping the original outputs
pickle.dump( num_y_test, open( "original_output_embeddings.p", "wb" ) )

In [28]:
# for efficient memory
del padded_embedding
#del training_dataset,test_dataset
del num_x_train
del num_y_train
del num_x_test

del x_test
del y_test